In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import cv2
import pickle

In [84]:
IMG_WIDTH=100
IMG_HEIGHT=100
img_folder=r'images/train_set/'

In [3]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, 0)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
            
    return img_data_array, class_name# extract the image array and class name




In [85]:
try:
    with open("train_dataset.pickle","rb") as f:
        img_data_inp,y=pickle.load(f)
except:
    img_data, class_name = create_dataset(img_folder)
    img_data = np.array(img_data)
    img_data_flat = np.reshape(img_data,(img_data.shape[0],IMG_HEIGHT*IMG_WIDTH))
    img_data_inp = img_data_flat.T
    print(img_data_inp.shape)
    
    class_name=np.array(class_name)
    class_name_inp=np.reshape(class_name,(1,class_name.shape[0]))
    print(class_name_inp.shape)

    x=img_data_inp
    y=np.zeros((1,x.shape[1]))
    for i in range(class_name_inp.shape[1]):
        if class_name[i]=='cats':
            y[0][i]=1
        else:
            y[0][i]=0 
    print(y.shape)
            
    with open("train_dataset.pickle","wb") as f:
        pickle.dump((img_data_inp,y),f)

input shapes 

img_data_array = 8000 images of size 50*50

[X1 X2 X3........X8000] #each column is an image of size 2500

class_name = 8000 values of corresponding images

[cat/dog  cat/dog ......... cat/dog] #length = 8000


# 1 layer NN

In [12]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [13]:
def layer_sizes(x,y):
    n_x=x.shape[0]
    n_h=4
    n_y=y.shape[0]
    
    return n_x,n_h,n_y

In [14]:
def initialize_parameters(n_x,n_h,n_y):
    
    w1=np.random.randn(n_h,n_x)
    b1=np.zeros(n_h,1)
    w2=np.random(n_y,n_h)
    b2=np.zeros(n_y,1)
    params ={"w1":w1,"b1":b1,"w2":w2,"b2":b2}
    return params

In [11]:
def forward_propogation(x,parameters):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    
    z1=np.dot(w1,x)+b1
    A1=np.tanh(z1)
    z2=np.dot(w2,A1)+b2
    A2=sigmoid(z2)
    
    cache={"A1":A1,"A2":A2}
    return cache

In [12]:
def compute_cost(A2,y,parameters):
    cost_mat=np.multiply(y,np.log(A2))+npmultiply((1-y),np.log(1-A2))
    cost=np.sum(cost_mat)/(-y.shape[1])
    
    cost=float(np.squeeze(cost))
    assert(isinstance(cost,float))
    return cost

In [13]:
def initialize_zeros(n_features):
    w=np.zeros((n_features,1))
    b=0
    return w,b

In [14]:
epsilon = 1e-5    
def propogate(w,b,x,y):
    m=x.shape[1]
    A = sigmoid(np.dot(w.T,x)+b)
    cost=(-1/m)*np.sum(y*np.log(A+epsilon)+(1-y)*np.log(1-A+epsilon))
    dw=(1/m)*np.dot(x,(A-y).T)
    db=(1/m)*np.sum(A-y)
    grads={"dw":dw,"db":db}
    #print(dw)
    return grads,cost

In [15]:
def optimize(w,b,x,y,n_iter,eta,batch_size,alpha):
    for i in range(n_iter):
        ind = np.random.choice(x.shape[1],batch_size)
        grads,cost=propogate(w,b,x[:,ind],y[:,ind])
        dw=grads["dw"]
        db=grads["db"]
        #print(dw)
        w=w-eta*dw
        #print(w)
        b=b-eta*db
        if(i%100==0):
            print(cost)
    params={"w":w,"b":b}
    grads={"dw":dw,"db":db}
    
    return params#,grads,cost

In [16]:
def predict(w,b,x):
    m=x.shape[1]
    y_predict=np.zeros((1,m))
    w=w.reshape(x.shape[0],1)
    
    A=sigmoid(np.dot(w.T,x)+b)
    for i in range(A.shape[1]):
        if(A[0][i]>0.5):
            y_predict[0,i]=1
        else:
            y_predict[0,i]=0
        
    return y_predict

In [17]:
        
w,b=initialize_zeros(x.shape[0])
params=optimize(w,b,x,y,10000,.01,100,0.9)

0.6931271807599428
0.6818079034702805
0.6941404280222065
0.7725678179166853
0.698797863299633
0.7322987838794961
0.6748007383658583
0.6711529517363966
0.678921989202119
0.6556643105754608
0.7395659863869033
0.6565945953167911
0.6518947728418146
0.6310425895172462
0.6821151831971314
0.677185770383172
0.7306790856758559
0.6569837847400064
0.7125411276607291
0.6782642238394898
0.6458893682205278
0.6837318616621759
0.7086906808312976
0.6676041042298719
0.6540237948851545
0.6695271960994529
0.6664831776101118
0.6656708167213831
0.6595026090334684
0.6495709385889106
0.6592413338382198
0.6767932608758281
0.6465300686772588
0.7000244299907235
0.6760495592028203
0.6523791363655279
0.6852646274119601
0.6528588238815265
0.7110372910579753
0.654013643430263
0.6847405687533891
0.6611121562379465
0.6941534082020671
0.7134354733633748
0.6882080630291404
0.6863887614907256
0.6576114332606247
0.6252093137996136
0.6521319531611546
0.6531916306884893
0.656770186165197
0.6584728481951219
0.674651283076835

LOSS VALUE    -       ACCURACY  <br>
       1.0   ------------             0.36  <br>
       0.9   ------------             0.40<br>
       0.8   ------------             0.44<br>
       0.7   ------------             0.49<br>
       0.6   ------------             0.54<br>
       0.5   ------------             0.60<br>
       0.4   ------------             0.67<br>
       0.3   ------------             0.74<br>
       0.2    ------------            0.81<br>

# 2 layer NN

In [15]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [16]:
def layer_sizes(x,y):
    n_x = x.shape[0]
    n_h = 4
    n_y = y.shape[0]
    return n_x,n_h,n_y


In [17]:
def initialize_parameters(n_x,n_h,n_y):
    
    w1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    w2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    parameters={"w1":w1,"b1":b1,"w2":w2,"b2":b2}
    return parameters


In [18]:
def forward_prop(x,parameters):
    w1=parameters["w1"]
    w2=parameters["w2"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    
    z1=np.dot(w1,x)+b1
    A1=np.tanh(z1)
    z2=np.dot(w2,A1)+b2
    A2=sigmoid(z2)
    
    cache={"A1":A1,"A2":A2}
    return cache


In [19]:
def compute_cost(A2,y):
    m=y.shape[1]
    cost_mat=np.multiply(y,np.log(A2))+np.multiply((1-y),np.log(1-A2))
    cost=np.sum(cost_mat)/(-m)
    cost=float(np.squeeze(cost))
    return cost

    

In [20]:
def compute_grad(x,parameters,cache,y):
    
    # X in dw1 calculation
    # w2 in dz1 calculation
    # A2,A1 in dz1,dw2 calculations
    #y in dz2 calculation
    
    m=x.shape[1]
    
    w1=parameters["w1"]
    w2=parameters["w2"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    
    A1=cache["A1"]
    A2=cache["A2"]
    
    dz2 = A2-y
    dw2=1/m*np.dot(dz2,A1.T)
    db2=1/m*np.sum(dz2,axis=1,keepdims=True)
    
    dz1=np.dot(w2.T,dz2)*(1-np.power(A1,2))
    dw1=1/m*np.dot(dz1,x.T)
    db1=1/m*np.sum(dz1,axis=1,keepdims=True)
    
    grads={"dw1":dw1,"dw2":dw2,"db1":db1,"db2":db2}
    
    return grads
    
    

In [21]:
def backward_prop(grads,parameters,eta):
    

    
    
    m=x.shape[1]
    
    w1=parameters["w1"]
    w2=parameters["w2"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    
    dw1=grads["dw1"]
    dw2=grads["dw2"]
    db1=grads["db1"]
    db2=grads["db2"]
    
    #grads={"dw1":dw1,"db1":db1,"dw2":dw2,"db2":db2}
    w1=w1-eta*dw1
    w2=w2-eta*dw2
    b1=b1-eta*db1
    b2=b2-eta*db2
    
    parameters={"w1":w1,"b1":b1,"w2":w2,"b2":b2}
    
    return parameters


In [27]:
'''def nn_model(x,y,n_h,n_iter=1000,eta=0.5,batch_size=100):
    np.random.seed(3)
    
    n_x=layer_sizes(x,y)[0]
    n_y=layer_sizes(x,y)[2]
    
    parameters=initialize_parameters(n_x,n_h,n_y)
    
    for i in range(n_iter+1):
        cache=forward_prop(x,parameters)
        cost=compute_cost(cache["A2"],y)
        grads=compute_grad(x,parameters,cache,y)
        parameters=backward_prop(grads,parameters,eta)
        if i%100==0:
            print("iteration ",i,"; cost ",cost,";")
    
    return parameters
    '''

In [ ]:
#parameters=nn_model(img_data_inp,y,n_h=4,n_iter=1000,eta=0.5,batch_size=100)

MINI BATCH GRADIENT

In [28]:
def nn_model(x,y,n_h,n_iter=1000,eta=0.5,batch_size=100):
    np.random.seed(3)
    

    n_x=layer_sizes(x,y)[0]
    n_y=layer_sizes(x,y)[2]
    
    parameters=initialize_parameters(n_x,n_h,n_y)

    #min_cost=100
    #min_parameters=parameters
    
    for i in range(n_iter+1):
        ind = np.random.choice(x.shape[1],batch_size)
        cache=forward_prop(x[:,ind],parameters)
        cost=compute_cost(cache["A2"],y[:,ind])   #HERE LOSS IS ONLY CALCULATED ON THE 'IND' SUBSET
        #if(cost<min_cost):
        #    min_cost=cost
        #    min_parameters=parameters
        grads=compute_grad(x[:,ind],parameters,cache,y[:,ind])
        parameters=backward_prop(grads,parameters,eta)
        if i%100==0:
            print("iteration ",i,"; cost ",cost,";")
    
    return parameters

In [29]:
parameters=nn_model(img_data_inp,y,n_h=4,n_iter=200000,eta=0.005,batch_size=50)

iteration  0 ; cost  0.6927997933556272 ;
iteration  100 ; cost  0.693457374797393 ;
iteration  200 ; cost  0.6952461028949958 ;
iteration  300 ; cost  0.6934480691608884 ;
iteration  400 ; cost  0.6923045720597518 ;
iteration  500 ; cost  0.6916354127253347 ;
iteration  600 ; cost  0.6926890769556011 ;
iteration  700 ; cost  0.6926542556066982 ;
iteration  800 ; cost  0.6911151851259336 ;
iteration  900 ; cost  0.6913024412396024 ;
iteration  1000 ; cost  0.6900385285433732 ;
iteration  1100 ; cost  0.6911802432299797 ;
iteration  1200 ; cost  0.6877157976951039 ;
iteration  1300 ; cost  0.6861933853603476 ;
iteration  1400 ; cost  0.6943864808032859 ;
iteration  1500 ; cost  0.689125736221599 ;
iteration  1600 ; cost  0.6768326615209221 ;
iteration  1700 ; cost  0.6835677420597012 ;
iteration  1800 ; cost  0.6837411510175124 ;
iteration  1900 ; cost  0.6664308254995914 ;
iteration  2000 ; cost  0.6901627847415195 ;
iteration  2100 ; cost  0.696668352186971 ;
iteration  2200 ; cost  0

iteration  18100 ; cost  0.5507720970655813 ;
iteration  18200 ; cost  0.6699544621512575 ;
iteration  18300 ; cost  0.552825234452839 ;
iteration  18400 ; cost  0.6452183693930919 ;
iteration  18500 ; cost  0.6213143056145689 ;
iteration  18600 ; cost  0.5284535240585472 ;
iteration  18700 ; cost  0.5873117778024338 ;
iteration  18800 ; cost  0.5722237760043553 ;
iteration  18900 ; cost  0.5297646228871749 ;
iteration  19000 ; cost  0.5873949364196115 ;
iteration  19100 ; cost  0.5346712141163574 ;
iteration  19200 ; cost  0.5194908913484629 ;
iteration  19300 ; cost  0.5332456846424858 ;
iteration  19400 ; cost  0.4770917779908019 ;
iteration  19500 ; cost  0.6371201616536876 ;
iteration  19600 ; cost  0.5222124586771945 ;
iteration  19700 ; cost  0.5759517747183709 ;
iteration  19800 ; cost  0.6001845774641389 ;
iteration  19900 ; cost  0.5624272275551381 ;
iteration  20000 ; cost  0.6199257292224435 ;
iteration  20100 ; cost  0.5295945483173685 ;
iteration  20200 ; cost  0.58220326

iteration  36000 ; cost  0.645311984876633 ;
iteration  36100 ; cost  0.5875008024352874 ;
iteration  36200 ; cost  0.5145353954918795 ;
iteration  36300 ; cost  0.5275406348317071 ;
iteration  36400 ; cost  0.4052087300941387 ;
iteration  36500 ; cost  0.5245723670358657 ;
iteration  36600 ; cost  0.4642562601999303 ;
iteration  36700 ; cost  0.3991165372352109 ;
iteration  36800 ; cost  0.4807777109480502 ;
iteration  36900 ; cost  0.682382552616949 ;
iteration  37000 ; cost  0.44963393391355333 ;
iteration  37100 ; cost  0.48798607433072566 ;
iteration  37200 ; cost  0.4988556465374858 ;
iteration  37300 ; cost  0.5038818280889809 ;
iteration  37400 ; cost  0.6461817912192467 ;
iteration  37500 ; cost  0.5535582104179456 ;
iteration  37600 ; cost  0.5057141238549208 ;
iteration  37700 ; cost  0.4171931922896009 ;
iteration  37800 ; cost  0.4884564755000151 ;
iteration  37900 ; cost  0.42233094996223997 ;
iteration  38000 ; cost  0.5199589841524719 ;
iteration  38100 ; cost  0.414303

iteration  53800 ; cost  0.42559400344801057 ;
iteration  53900 ; cost  0.9182968958137245 ;
iteration  54000 ; cost  0.4852047707416948 ;
iteration  54100 ; cost  0.43494055255693054 ;
iteration  54200 ; cost  0.5161135950400838 ;
iteration  54300 ; cost  0.3005068896528812 ;
iteration  54400 ; cost  0.4392408371551035 ;
iteration  54500 ; cost  0.408486105326678 ;
iteration  54600 ; cost  0.24898945343081544 ;
iteration  54700 ; cost  0.4774095907261945 ;
iteration  54800 ; cost  0.3027854157901579 ;
iteration  54900 ; cost  0.8029665772230468 ;
iteration  55000 ; cost  0.37643468779283684 ;
iteration  55100 ; cost  0.40949306067018637 ;
iteration  55200 ; cost  0.41287624880947943 ;
iteration  55300 ; cost  0.43700383660542275 ;
iteration  55400 ; cost  0.5732295300554893 ;
iteration  55500 ; cost  0.7553344472823822 ;
iteration  55600 ; cost  0.36186508958341385 ;
iteration  55700 ; cost  0.3374020276498269 ;
iteration  55800 ; cost  0.43126147412400107 ;
iteration  55900 ; cost  0

iteration  71500 ; cost  0.40310196707538587 ;
iteration  71600 ; cost  0.3309119491821482 ;
iteration  71700 ; cost  0.348756591672343 ;
iteration  71800 ; cost  0.4868609141873836 ;
iteration  71900 ; cost  0.40497443418416845 ;
iteration  72000 ; cost  0.31600209615100655 ;
iteration  72100 ; cost  0.3510591487256008 ;
iteration  72200 ; cost  0.3590318618786468 ;
iteration  72300 ; cost  0.5743977037358077 ;
iteration  72400 ; cost  0.27889507872756886 ;
iteration  72500 ; cost  0.38051250022956234 ;
iteration  72600 ; cost  0.29725884362134464 ;
iteration  72700 ; cost  0.523189959049622 ;
iteration  72800 ; cost  0.4782335792171515 ;
iteration  72900 ; cost  0.3492883223849182 ;
iteration  73000 ; cost  0.3937388033463629 ;
iteration  73100 ; cost  0.3542967755387819 ;
iteration  73200 ; cost  0.3515372521790725 ;
iteration  73300 ; cost  0.25809357256569826 ;
iteration  73400 ; cost  0.5069582286734845 ;
iteration  73500 ; cost  0.40678741619336395 ;
iteration  73600 ; cost  0.2

...

iteration  124000 ; cost  0.2674260053570688 ;
iteration  124100 ; cost  0.22271616281567183 ;
iteration  124200 ; cost  0.23948952653580946 ;
iteration  124300 ; cost  0.23968140110946248 ;
iteration  124400 ; cost  0.7861721970126403 ;
iteration  124500 ; cost  0.2521510261401439 ;
iteration  124600 ; cost  0.23940092979852917 ;
iteration  124700 ; cost  0.6686729333371417 ;
iteration  124800 ; cost  0.17757598217132034 ;
iteration  124900 ; cost  0.2004590035492035 ;
iteration  125000 ; cost  0.3747588215164528 ;
iteration  125100 ; cost  0.15916252896030422 ;
iteration  125200 ; cost  0.21368005034952245 ;
iteration  125300 ; cost  0.1605221181064813 ;
iteration  125400 ; cost  0.17824455627555674 ;
iteration  125500 ; cost  0.17357489490281214 ;
iteration  125600 ; cost  0.2439224775389416 ;
iteration  125700 ; cost  0.20575827337200486 ;
iteration  125800 ; cost  0.36080916006498337 ;
iteration  125900 ; cost  0.34017821177884366 ;
iteration  126000 ; cost  0.2388632909782715 ;
i

iteration  141300 ; cost  0.2738014954510904 ;
iteration  141400 ; cost  0.2544926689119549 ;
iteration  141500 ; cost  0.7222109924339799 ;
iteration  141600 ; cost  0.17247457161963442 ;
iteration  141700 ; cost  0.18908249883984357 ;
iteration  141800 ; cost  0.0977768211765188 ;
iteration  141900 ; cost  0.1280282569525553 ;
iteration  142000 ; cost  0.15193292252025864 ;
iteration  142100 ; cost  0.34172173768914293 ;
iteration  142200 ; cost  0.32648107837579055 ;
iteration  142300 ; cost  0.22309589431736743 ;
iteration  142400 ; cost  0.1790093702278013 ;
iteration  142500 ; cost  0.11287096382451342 ;
iteration  142600 ; cost  0.6638830127972496 ;
iteration  142700 ; cost  0.08806984191768137 ;
iteration  142800 ; cost  0.4754905979822468 ;
iteration  142900 ; cost  0.2869636110359949 ;
iteration  143000 ; cost  0.21176573531508416 ;
iteration  143100 ; cost  0.12630081569051632 ;
iteration  143200 ; cost  0.13489473441515654 ;
iteration  143300 ; cost  0.2531918222199433 ;
it

iteration  158600 ; cost  0.4543452648508892 ;
iteration  158700 ; cost  0.43305497490335493 ;
iteration  158800 ; cost  0.404837090948426 ;
iteration  158900 ; cost  0.34792576980930723 ;
iteration  159000 ; cost  0.2685155328607932 ;
iteration  159100 ; cost  0.5315722237963441 ;
iteration  159200 ; cost  0.28024201902472046 ;
iteration  159300 ; cost  0.3563077848341367 ;
iteration  159400 ; cost  0.21940290293250367 ;
iteration  159500 ; cost  0.08932059789739485 ;
iteration  159600 ; cost  0.20426251027900147 ;
iteration  159700 ; cost  0.8629685712008863 ;
iteration  159800 ; cost  0.10881769843336561 ;
iteration  159900 ; cost  0.20618542050262878 ;
iteration  160000 ; cost  0.16390036307858932 ;
iteration  160100 ; cost  0.20058699862124402 ;
iteration  160200 ; cost  0.14209521082983217 ;
iteration  160300 ; cost  0.16064348589543548 ;
iteration  160400 ; cost  0.32645869263825267 ;
iteration  160500 ; cost  0.28728318597753044 ;
iteration  160600 ; cost  0.19184490793356168 ;

iteration  175900 ; cost  0.1608377068985505 ;
iteration  176000 ; cost  0.14183879333537044 ;
iteration  176100 ; cost  0.15654599063209232 ;
iteration  176200 ; cost  0.1327485139982447 ;
iteration  176300 ; cost  0.08768597848488462 ;
iteration  176400 ; cost  0.11446422868426886 ;
iteration  176500 ; cost  0.18061136281198395 ;
iteration  176600 ; cost  0.12815146090952956 ;
iteration  176700 ; cost  0.23017945607936774 ;
iteration  176800 ; cost  0.07380563887711063 ;
iteration  176900 ; cost  0.1806919967123141 ;
iteration  177000 ; cost  0.11976029907863543 ;
iteration  177100 ; cost  0.20935871520058189 ;
iteration  177200 ; cost  0.10292100024264472 ;
iteration  177300 ; cost  0.11166733612202547 ;
iteration  177400 ; cost  0.15818378191024338 ;
iteration  177500 ; cost  0.07763867242732825 ;
iteration  177600 ; cost  0.06603859464679317 ;
iteration  177700 ; cost  0.14089550966160888 ;
iteration  177800 ; cost  0.18101957838428212 ;
iteration  177900 ; cost  0.216477620267921

iteration  193100 ; cost  0.19219446609795682 ;
iteration  193200 ; cost  0.5414867786829728 ;
iteration  193300 ; cost  0.749447828141573 ;
iteration  193400 ; cost  0.264038666627908 ;
iteration  193500 ; cost  0.21610454020845224 ;
iteration  193600 ; cost  0.3479792163156311 ;
iteration  193700 ; cost  0.167737766100456 ;
iteration  193800 ; cost  0.07558144661477016 ;
iteration  193900 ; cost  0.20046246591500208 ;
iteration  194000 ; cost  0.0620963865373038 ;
iteration  194100 ; cost  0.17942808048695238 ;
iteration  194200 ; cost  0.2276838076699802 ;
iteration  194300 ; cost  0.08083736121803828 ;
iteration  194400 ; cost  0.20596567461142842 ;
iteration  194500 ; cost  0.17788970730118234 ;
iteration  194600 ; cost  0.18978861695248345 ;
iteration  194700 ; cost  0.09402146323564979 ;
iteration  194800 ; cost  0.11259675382208414 ;
iteration  194900 ; cost  0.18923970732773998 ;
iteration  195000 ; cost  0.04949395599047488 ;
iteration  195100 ; cost  0.1047722240252325 ;
ite

In [34]:
with open("final_param.pickle","wb") as f:
    pickle.dump((parameters),f)
    

In [57]:
cache=forward_prop(img_data_inp,parameters)
cost=compute_cost(cache["A2"],y)
cost

0.12705045487206607

In [69]:
def predict(parameters,x):
    A2=forward_prop(x,parameters)["A2"]
    #print(A2.shape)
    #predictions=np.zeros((1,x.shape[1]))
    predictions=A2>0.5
    return predictions

In [83]:
predictions = predict(parameters,x)
print("Accuracy : ",float(np.dot(y,predictions.T)+np.dot(1-y,1-predictions.T))/float(y.size)*100)#/float(y.size)*100)

Accuracy :  97.125


In [ ]:
def nn_model_2(x,y,n_h,n_iter=1000,eta=0.5,batch_size=100):
    np.random.seed(3)
    

    n_x=layer_sizes(x,y)[0]
    n_y=layer_sizes(x,y)[2]
    
    parameters=initialize_parameters(n_x,n_h,n_y)

    min_cost=100
    min_parameters=parameters
    
    for i in range(n_iter+1):
        ind = np.random.choice(x.shape[1],batch_size)
        cache=forward_prop(x[:,ind],parameters)
        #cost=compute_cost(cache["A2"],y[:,ind])   #HERE LOSS IS CALCULATED ON THE ENTIRE INPUT
        #if(cost<min_cost):
            #min_cost=cost
            #min_parameters=parameters
        grads=compute_grad(x[:,ind],parameters,cache,y[:,ind])
        parameters=backward_prop(grads,parameters,eta)
        if i%1000==0:
            cache=forward_prop(img_data_inp,parameters)
            cost=compute_cost(cache["A2"],y)
            print("iteration ",i,"; cost ",cost,";")
    
    return min_parameters

In [60]:
def predict(parameters,x):
    A2=forward_prop(x,parameters)["A2"]
    return A2

In [90]:
count=0
for i in range(4001,5001):
    filename = "images/test_set/dogs/dog." + str(i) + ".jpg"
    image = cv2.imread(filename,0)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image /= 255 
    image_flatten = np.reshape(image,(IMG_HEIGHT*IMG_WIDTH,1))
    res=predict(parameters,image_flatten)
    if(res[0][0]<0.5):
        count+=1

In [91]:
count

557

In [86]:
count=0
for i in range(4001,5001):
    filename = "images/test_set/cats/cat." + str(i) + ".jpg"
    image = cv2.imread(filename,0)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image /= 255 
    image_flatten = np.reshape(image,(IMG_HEIGHT*IMG_WIDTH,1))
    res=predict(parameters,image_flatten)
    if(res[0][0]>0.5):
        count+=1

In [87]:
count

591